<a href="https://colab.research.google.com/github/6Santiago9/Entregas/blob/main/04%20-%20Entrenamiento%20con%20una%20Red%20Neuronal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Red Neuronal
En el segundo entrenamiento use este modelo con una limpieza diferente

# Importación de Librerías

In [ ]:
import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import unicodedata
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Configuración Kaggle

In [ ]:
# Establecer tus credenciales de Kaggle
os.environ['KAGGLE_USERNAME'] = 'cesarmartinezia'
os.environ['KAGGLE_KEY'] = '3bd3c2a5994356c24295cd5c6d8bba59'

In [ ]:
# Crear manualmente el archivo kaggle.json a partir de esas variables
!mkdir -p ~/.kaggle
with open('/root/.kaggle/kaggle.json', 'w') as f:
    f.write('{"username":"%s","key":"%s"}' % (os.environ['KAGGLE_USERNAME'], os.environ['KAGGLE_KEY']))

In [ ]:
# Ajustar permisos
!chmod 600 /root/.kaggle/kaggle.json

# Descarga de Dataset

In [ ]:
# Descargar los datos de la competencia
!kaggle competitions download -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia

udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip

Archive:  udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip
replace submission_example.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

# Cargar el train y test

In [ ]:
df = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [ ]:
testID = test.copy()

In [ ]:
# Tamaño del dataset
print(df.shape, test.shape)

(692500, 21) (296786, 20)


# Limpieza tanto en test como train

In [ ]:
# Revisar valores faltantes
df.isnull().sum().sort_values(ascending=False).head(10)
test.isnull().sum().sort_values(ascending=False).head(10)
# Si es categórica reemplazar con la moda (valor más frecuente)
for col in df.select_dtypes(include=['object']):
    df[col].fillna(df[col].mode()[0], inplace=True)
for col in test.select_dtypes(include=['object']):
    test[col].fillna(test[col].mode()[0], inplace=True)

/tmp/ipython-input-4292607415.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mode()[0], inplace=True)
/tmp/ipython-input-4292607415.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try usi

## Columnas duplicadas

In [ ]:
df.drop('F_TIENEINTERNET.1', axis=1, inplace=True)
test.drop('F_TIENEINTERNET.1', axis=1, inplace=True)

## Palabras mal escritas o con mucha similitud


In [ ]:
# Función para eliminar tildes y normalizar texto
def normalizar_texto(texto):
    if isinstance(texto, str):
        # Quitar tildes (por ejemplo: Á → A)
        texto = ''.join(
            c for c in unicodedata.normalize('NFD', texto)
            if unicodedata.category(c) != 'Mn'
        )
        # Convertir a mayúsculas
        texto = texto.strip().upper()
        return texto
    else:
        return texto
# Aplicar la función directamente sobre la columna original
df['E_PRGM_ACADEMICO'] = df['E_PRGM_ACADEMICO'].apply(normalizar_texto)
test['E_PRGM_ACADEMICO'] = test['E_PRGM_ACADEMICO'].apply(normalizar_texto)
# Reemplazamos el nombre de algunas variables
df["E_PRGM_ACADEMICO"] = df["E_PRGM_ACADEMICO"].str.replace('DISENO','DISEÑO', regex=False)
test["E_PRGM_ACADEMICO"] = test["E_PRGM_ACADEMICO"].str.replace('DISENO','DISEÑO', regex=False)
df["E_PRGM_ACADEMICO"] = df["E_PRGM_ACADEMICO"].str.replace('LICENCIATURA M¿SICA','LICENCIATURA', regex=False)
test["E_PRGM_ACADEMICO"] = test["E_PRGM_ACADEMICO"].str.replace('LICENCIATURA M¿SICA','LICENCIATURA', regex=False)

Eliminar espacios

In [ ]:
df['E_PRGM_ACADEMICO'] = df['E_PRGM_ACADEMICO'].apply(lambda x: re.sub(r'\s+', ' ', x.strip()) if isinstance(x, str) else x)
test['E_PRGM_ACADEMICO'] = test['E_PRGM_ACADEMICO'].apply(lambda x: re.sub(r'\s+', ' ', x.strip()) if isinstance(x, str) else x)

Ahora unimos todos los programas que tienen la misma similitud

In [ ]:
pip install thefuzz[speedup]

In [ ]:
from thefuzz import process
# -- Selecciona la columna que quieres agrupar ---
columna = 'E_PRGM_ACADEMICO'
# -- Define las categorías limpias (referencias) ---
categorias_referencia = [
    'ADMINISTRACION DE EMPRESAS',
    'ADMINISTRACION',
    'COMERCIO',
    'ARTES',
    'BIOLOGIA',
    'CIENCIAS',
    "CINE",
    "COMUNICACION",
    'CONTADURIA',
    "DERECHO",
    "DISENO",
    "INGENIERIA DE SISTEMAS",
    "INGENIERIA CIVIL",
    "INGENIERIA DE MINAS",
    "INGENIERIA ELECTRICA",
    "INGENIERIA INDUSTRIAL",
    "INGENIERIA MECANICA",
    "INGENIERIA QUIMICA",
    "MERCADEO",
    "LICENCIATURA",
    "NEGOCIOS INTERNACIONALES",
    "QUIMICA",
    "GASTRONOMIA"
]
#  Aplica el agrupamiento difuso
def corregir_categoria(valor):
    if pd.isnull(valor):
        return valor
    mejor_match = process.extractOne(valor, categorias_referencia)
    if mejor_match[1] >= 70:  # porcentaje mínimo de similitud
        return mejor_match[0]
    else:
        return valor
df[columna] = df[columna].apply(corregir_categoria)
test[columna] = test[columna].apply(corregir_categoria)

# Preprosesado

## Convertir variables categoricas en numericas

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['F_TIENELAVADORA'] = le.fit_transform(df['F_TIENELAVADORA']) # Aqui el 1 es si y 0 es no
df['F_TIENEINTERNET'] = le.fit_transform(df['F_TIENEINTERNET']) # Aqui el 1 es si y 0 es no
df['F_TIENEAUTOMOVIL'] = le.fit_transform(df['F_TIENEAUTOMOVIL']) # Aqui el 1 es si y 0 es no
df['E_PRIVADO_LIBERTAD'] = le.fit_transform(df['E_PRIVADO_LIBERTAD']) # Aqui el 1 es si y 0 es no
df['E_PAGOMATRICULAPROPIO'] = le.fit_transform(df['E_PAGOMATRICULAPROPIO']) # Aqui el 1 es si y 0 es no
df['F_TIENECOMPUTADOR'] = le.fit_transform(df['F_TIENECOMPUTADOR']) # Aqui el 1 es s y 0 es n
test['F_TIENELAVADORA'] = le.fit_transform(test['F_TIENELAVADORA']) # Aqui el 1 es si y 0 es no
test['F_TIENEINTERNET'] = le.fit_transform(test['F_TIENEINTERNET']) # Aqui el 1 es si y 0 es no
test['F_TIENEAUTOMOVIL'] = le.fit_transform(test['F_TIENEAUTOMOVIL']) # Aqui el 1 es si y 0 es no
test['E_PRIVADO_LIBERTAD'] = le.fit_transform(test['E_PRIVADO_LIBERTAD']) # Aqui el 1 es si y 0 es no
test['E_PAGOMATRICULAPROPIO'] = le.fit_transform(test['E_PAGOMATRICULAPROPIO']) # Aqui el 1 es si y 0 es no
test['F_TIENECOMPUTADOR'] = le.fit_transform(test['F_TIENECOMPUTADOR']) # Aqui el 1 es s y 0 es n

Aqui reemplazamos variables categoricas que contienen mas de un nivel

In [ ]:
# En estrato solo se puso el numero al que pertenecian.
estrato = {'Sin Estrato' : 0,
    'Estrato 1' : 1,
    'Estrato 2' : 2,
    'Estrato 3' : 3,
    'Estrato 4' : 4,
    'Estrato 5' : 5,
    'Estrato 6' : 6}
# En trabaja se puso un valor que estara en el rango de la descripción dada
trabaja = {"Más de 30 horas" : 3,
           "0" : 0,
           "Entre 11 y 20 horas" : 1,
           "Entre 21 y 30 horas" : 2,
           "Menos de 10 horas" : 4}
# En matricula tambien se puso un valor que estara entre el valor menconado.
matricula = {"Entre 1 millón y menos de 2.5 millones" : 2,
             "Entre 2.5 millones y menos de 4 millones" : 3,
             "Menos de 500 mil" : 1,
             "Entre 500 mil y menos de 1 millón" : 7,
             "Entre 4 millones y menos de 5.5 millones" : 5,
             "Más de 7 millones" : 8,
             "Entre 5.5 millones y menos de 7 millones" : 6,
             "No pagó matrícula" : 0}
df['E_VALORMATRICULAUNIVERSIDAD'].replace(matricula, inplace=True)
df['E_HORASSEMANATRABAJA'].replace(trabaja, inplace=True)
df['F_ESTRATOVIVIENDA'].replace(estrato, inplace=True)
test['E_VALORMATRICULAUNIVERSIDAD'].replace(matricula, inplace=True)
test['E_HORASSEMANATRABAJA'].replace(trabaja, inplace=True)
test['F_ESTRATOVIVIENDA'].replace(estrato, inplace=True)

/tmp/ipython-input-3863103679.py:24: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['E_VALORMATRICULAUNIVERSIDAD'].replace(matricula, inplace=True)
/tmp/ipython-input-3863103679.py:24: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['E_VALORMATRICULAUNIVERSIDAD'].replace(matricula, inplace=True)


## Utilizar One-HOt

In [ ]:
# ==================================================
# 1. One-Hot Encoding seguro y consistente
# ==================================================

# Unir train y test temporalmente para crear dummies iguales
df["is_train"] = 1
test["is_train"] = 0

full = pd.concat([df, test], axis=0)

# Lista de columnas categóricas
cat_cols = [
    'E_PRGM_DEPARTAMENTO',
    'F_EDUCACIONMADRE',
    'F_EDUCACIONPADRE',
    'E_PRGM_ACADEMICO'
]

# Aplicar dummies una sola vez
full = pd.get_dummies(full, columns=cat_cols, dtype=int)

# Volver a separar
df = full[full["is_train"] == 1].drop("is_train", axis=1)
test  = full[full["is_train"] == 0].drop("is_train", axis=1)

print("✔ Dummies iguales para train y test")
print(df.shape, test.shape)

✔ Dummies iguales para train y test
(692500, 225) (296786, 225)


In [ ]:
train = df.copy()
test = test.copy()

## Entrenamiento

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
if "RENDIMIENTO_GLOBAL" in test.columns:
    test = test.drop("RENDIMIENTO_GLOBAL", axis=1)

# Alinear columnas (muy importante)
test = test.reindex(columns=train.drop("RENDIMIENTO_GLOBAL", axis=1).columns, fill_value=0)
# ==================================================
# Separar X e y
# ==================================================
X = train.drop("RENDIMIENTO_GLOBAL", axis=1)
y = train["RENDIMIENTO_GLOBAL"]

test_IDs = test['ID']
X_test = test.drop(columns=['ID'])
# One-hot para Keras
y_cat = pd.get_dummies(y)

# ==================================================
# Train/Valid split
# ==================================================
X_train, X_valid, y_train, y_valid = train_test_split(
    X, y_cat, test_size=0.2, random_state=42, stratify=y
)

print("Train:", X_train.shape, "Valid:", X_valid.shape)

# ==================================================
# Normalización
# ==================================================
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
test_scaled    = scaler.transform(test)

# ==================================================
# Modelo Keras (profundo pero ligero)
# ==================================================
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.3),

    Dense(64, activation='relu'),
    Dropout(0.2),

    Dense(32, activation='relu'),

    Dense(y_cat.shape[1], activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

# ==================================================
# Early Stopping (RAM eficiente)
# ==================================================
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

# ==================================================
# Entrenamiento
# ==================================================
history = model.fit(
    X_train_scaled, y_train,
    validation_data=(X_valid_scaled, y_valid),
    epochs=15,
    batch_size=512,
    callbacks=[early_stop],
    verbose=1
)

# ==================================================
# Evaluación
# ==================================================
loss, acc = model.evaluate(X_valid_scaled, y_valid)
print("📌 Accuracy en validación:", acc)

# ==================================================
# Predicción para test
# ==================================================
pred_test = model.predict(test_scaled)
pred_labels = y_cat.columns[np.argmax(pred_test, axis=1)]

# ==================================================
# Submission
# ==================================================
submission = pd.DataFrame({
    "ID": test_IDs,
    "RENDIMIENTO_GLOBAL": pred_labels
})

submission.to_csv("submission_keras.csv", index=False)
submission.head()

print("📁 Archivo 'submission_keras.csv' generado correctamente.")


Train: (554000, 224) Valid: (138500, 224)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │        28,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 4)              │           132 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 39,268 (153.39 KB)

 Trainable params: 39,268 (153.39 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/15
1083/1083 ━━━━━━━━━━━━━━━━━━━━ 15s 11ms/step - accuracy: 0.3663 - loss: 1.3045 - val_accuracy: 0.3993 - val_loss: 1.2563
Epoch 2/15
1083/1083 ━━━━━━━━━━━━━━━━━━━━ 11s 11ms/step - accuracy: 0.3995 - loss: 1.2567 - val_accuracy: 0.4119 - val_loss: 1.2384
Epoch 3/15
1083/1083 ━━━━━━━━━━━━━━━━━━━━ 11s 10ms/step - accuracy: 0.4079 - loss: 1.2428 - val_accuracy: 0.4155 - val_loss: 1.2313
Epoch 4/15
1083/1083 ━━━━━━━━━━━━━━━━━━━━ 21s 10ms/step - accuracy: 0.4134 - loss: 1.2372 - val_accuracy: 0.4182 - val_loss: 1.2286
Epoch 5/15
1083/1083 ━━━━━━━━━━━━━━━━━━━━ 11s 10ms/step - accuracy: 0.4152 - loss: 1.2326 - val_accuracy: 0.4200 - val_loss: 1.2273
Epoch 6/15
1083/1083 ━━━━━━━━━━━━━━━━━━━━ 11s 10ms/step - accuracy: 0.4182 - loss: 1.2283 - val_accuracy: 0.4228 - val_loss: 1.2247
Epoch 7/15
1083/1083 ━━━━━━━━━━━━━━━━━━━━ 19s 9ms/step - accuracy: 0.4192 - loss: 1.2273 - val_accuracy: 0.4215 - val_loss: 1.2255
Epoch 8/15
1083/1083 ━━━━━━━━━━━━━━━━━━━━ 12s 11ms/step - accuracy: 0.4201 - 